In [1]:
import pandas as pd
import numpy as np

from src.data_prep import shuffle_fighters, win_method_binner, pivot_longer, calculate_running_totals

pd.set_option('display.max_columns', None)

In [2]:
event_details = pd.read_csv('https://github.com/SamEdwardes/ufc-data/raw/master/event_details.csv')
event_details['date'] = pd.to_datetime(event_details['date'])
fighter_details = pd.read_csv('https://github.com/SamEdwardes/ufc-data/raw/master/fighter_details.csv')

In [3]:
df = shuffle_fighters(event_details)
df['win_method_bin'] = df['win_method'].apply(win_method_binner)
df_long = pivot_longer(df)
df_long = calculate_running_totals(df_long)
df_long = pd.merge(
    df_long, 
    fighter_details[['fighter_url', 'height_inches', 'weight_lbs', 'reach_inches', 'stance']], 
    how='left',
    left_on='fighter_url', right_on='fighter_url'
)
features_to_merge = [
    'fight_id', 'num_fights', 'wins', 'losses', 'days_since_last_fight', 
    'tko_recieved', 'total_tko_recieved', 'fight_time', 'total_octagon_time',
    'last_fight_time', 'last_fight_tko_received', 'last_fight_win',
    'last_fight_loss', 'height_inches', 'weight_lbs', 'reach_inches',
    'stance'
]

X = pd.merge(
    df,
    df_long.query('colour == "blue"')[features_to_merge].add_prefix('blue_'),
    left_on = 'fight_id',
    right_on = 'blue_fight_id'
)

X = pd.merge(
    X,
    df_long.query('colour == "red"')[features_to_merge].add_prefix('red_'),
    left_on = 'fight_id',
    right_on = 'red_fight_id'
)

In [4]:
X

,event_name,date,weight_class,win_method,win_round,win_time,fight_url,fight_id,blue,red,winner_colour,blue_url,red_url,winner_name,blue_str,red_str,blue_td,red_td,blue_sub,red_sub,blue_pass,red_pass,blue_win,win_method_bin,blue_fight_id,blue_num_fights,blue_wins,blue_losses,blue_days_since_last_fight,blue_tko_recieved,blue_total_tko_recieved,blue_fight_time,blue_total_octagon_time,blue_last_fight_time,blue_last_fight_tko_received,blue_last_fight_win,blue_last_fight_loss,blue_height_inches,blue_weight_lbs,blue_reach_inches,blue_stance,red_fight_id,red_num_fights,red_wins,red_losses,red_days_since_last_fight,red_tko_recieved,red_total_tko_recieved,red_fight_time,red_total_octagon_time,red_last_fight_time,red_last_fight_tko_received,red_last_fight_win,red_last_fight_loss,red_height_inches,red_weight_lbs,red_reach_inches,red_stance
0,UFC_Fight_Night-Overeem_vs_Harris,2020-05-16,Heavyweight,"KO/TKO, Punches",2,3:00,http://www.ufcstats.com/fight-details/ce074851...,0,Alistair Overeem,Walt Harris,blue,http://www.ufcstats.com/fighter-details/b4ad3a...,http://www.ufcstats.com/fighter-details/03f982...,Alistair Overeem,68.0,25.0,1.0,0.0,0.0,0.0,3.0,0.0,1,TKO,0,18,11,7,161.0,0,7,8.000000,150.616667,24.083333,1,0,1,76.0,265.0,80.0,Orthodox,0,14,6,8,301.0,1,3,8.000000,88.466667,0.016667,0,1,0,77.0,250.0,77.0,Southpaw
1,UFC_Fight_Night-Overeem_vs_Harris,2020-05-16,Women's Strawweight,S-DEC,3,5:00,http://www.ufcstats.com/fight-details/27bfb920...,1,Claudia Gadelha,Angela Hill,blue,http://www.ufcstats.com/fighter-details/ec3434...,http://www.ufcstats.com/fighter-details/f0feeb...,Claudia Gadelha,84.0,90.0,1.0,0.0,0.0,0.0,1.0,0.0,1,DEC,1,11,7,4,315.0,0,0,15.000000,163.000000,15.000000,0,1,0,64.0,115.0,63.0,Orthodox,1,15,7,8,84.0,0,0,15.000000,191.216667,15.000000,0,1,0,63.0,115.0,64.0,Orthodox
2,UFC_Fight_Night-Overeem_vs_Harris,2020-05-16,Featherweight,S-DEC,3,5:00,http://www.ufcstats.com/fight-details/58540b24...,2,Dan Ige,Edson Barboza,blue,http://www.ufcstats.com/fighter-details/82a515...,http://www.ufcstats.com/fighter-details/64a50d...,Dan Ige,79.0,80.0,1.0,0.0,0.0,0.0,0.0,0.0,1,DEC,2,7,6,1,98.0,0,0,15.000000,76.100000,15.000000,0,1,0,67.0,145.0,71.0,Orthodox,2,23,14,9,252.0,0,3,15.000000,255.416667,15.000000,0,0,1,71.0,145.0,75.0,Orthodox
3,UFC_Fight_Night-Overeem_vs_Harris,2020-05-16,Middleweight,U-DEC,3,5:00,http://www.ufcstats.com/fight-details/821a1c6f...,3,Krzysztof Jotko,Eryk Anders,blue,http://www.ufcstats.com/fighter-details/4e0691...,http://www.ufcstats.com/fighter-details/cad244...,Krzysztof Jotko,66.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DEC,3,13,9,4,294.0,0,2,15.000000,163.216667,15.000000,0,1,0,73.0,185.0,77.0,Southpaw,3,10,5,5,217.0,0,1,15.000000,132.166667,15.000000,0,1,0,73.0,185.0,75.0,Southpaw
4,UFC_Fight_Night-Overeem_vs_Harris,2020-05-16,Featherweight,U-DEC,3,5:00,http://www.ufcstats.com/fight-details/81f2f2d4...,4,Song Yadong,Marlon Vera,blue,http://www.ufcstats.com/fighter-details/efb96b...,http://www.ufcstats.com/fighter-details/7c7332...,Song Yadong,101.0,92.0,0.0,2.0,0.0,0.0,0.0,1.0,1,DEC,4,6,5,1,161.0,0,0,15.000000,60.100000,15.000000,0,0,1,68.0,145.0,67.0,Orthodox,4,14,9,5,217.0,0,0,15.000000,149.333333,13.016667,0,1,0,68.0,145.0,70.0,Switch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,UFC_1-The_Beginning,1993-11-12,Open Weight,KO/TKO,1,0:59,http://www.ufcstats.com/fight-details/ac7ca2ec...,5591,Gerard Gordeau,Kevin Rosier,blue,http://www.ufcstats.com/fighter-details/279093...,http://www.ufcstats.com/fighter-details/598a58...,Gerard Gordeau,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,TKO,5591,2,1,1,0.0,0,0,0.083333,1.150000,1.066667,0,0,1,77.0,216.0,NaN,Orthodox,5591,1,0,1,0.0,1,1,0.083333,0.083333,0.000000,0,0,0,76.0,275.0,NaN,Orthodox
5592,UFC_1-The_Beginning,1993-11-12,Open Weight,"SUB, Heel, Hook",1,1:49,http://www.ufcstats.com/